# Neural Nets t3

In [33]:
%matplotlib widget
#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# import Importing_Notebooks
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import dill

A network built of components which:
1. accept an ordered set of reals (we'll use `numpy.array`, and  call them vectors) at the input port and produce another at the output port - this is forward propagation. ${\displaystyle f\colon \mathbf {R} ^{n}\to \mathbf {R} ^{m}}$
1. accept an ordered set of reals at the output port, representing the gradient of the loss function at the output, and produce the gradient of the loss function at the input port - this is back propagation, aka backprop. ${\displaystyle b\colon \mathbf {R} ^{m}\to \mathbf {R} ^{n}}$
1. from the gradient of the loss function at the output, calculate the partial of the loss function w.r.t the internal parameters ${\displaystyle \frac{\partial E}{\partial w} }$
1. accept a scalar $\eta$ to control the adjustment of internal parameters. _Or is this effected by scaling the loss gradient before passing??_
1. update internal parameters ${\displaystyle w \leftarrow w - \eta \frac{\partial E}{\partial w} }$


In [35]:
class Layer:
    def __init__(self):
        pass
    
    def __call__(self, x):
        """Computes response to input"""
        raise NotImplementedError
        
    def backprop(self, output_delE):
        """Uses output error gradient to adjust internal parameters, and returns gradient of error at input"""
        raise NotImplementedError

A network built of a cascade of layers:

In [36]:
class Network:
    def __init__(self):
        self.layers = []
        self.eta = 0.1 #FIXME
        
    def extend(self, net):
        self.layers.append(net)
        
    def __call__(self, input):
        v = input
        for net in self.layers:
            v = net(v)
        return v
    
    def learn(self, facts):
        for (x, expected) in facts:
            y = self(x)
            e = y - expected
            loss = e.dot(e)/2.0
            egrad = e * self.eta
            for net in reversed(self.layers):
                egrad = net.backprop(egrad)
        return loss

## A Neural Net lab bench

In [37]:
# from nnbench import NNBench
from matplotlib.widgets import Slider, Button, RadioButtons

In [38]:
class NNBench:
    def __init__(self, net, ideal=lambda x:x):
        self.net = net
        self.ideal = ideal
        self.gc_protect = []
        self.seed = 3
    
    def checkpoint_net(self):
        self.net_checkpoint = dill.dumps(self.net)
        
    def rollback_net(self):
        self.net = dill.loads(self.net_checkpoint)
        
    def training_data_gen(self, n):
        """Generate n instances of labelled training data"""
        np.random.seed(self.seed)
        for i in range(n):
            v = np.random.randn(2)
            yield (v, self.ideal(v))
            
    def learn(self, n=100):
        return [self.net.learn([fact]) for fact in self.training_data_gen(n)]
            
    def learning_potential(self, n=100, eta=None):
        stash = dill.dumps(self.net)
        if eta is not None: # only change the net's eta if a value was passed to us
            self.net.eta = eta
        loss = self.net.learn(fact for fact in self.training_data_gen(n))
        self.net = dill.loads(stash)
        return -np.log(loss)
        
    def plot_learning(self, n):
        from matplotlib import pyplot as plt
        # self.losses = losses = [self.net.learn(fact for fact in self.training_data_gen(n))]
        losses = self.learn(n)
        plt.yscale('log')
        plt.plot(range(len(losses)),losses)
        plt.show(block=0)
        
    def knobs_plot_learning(self, n):
        pickled_net = dill.dumps(self.net)
        # from matplotlib import pyplot as plt
        fig, ax = plt.subplots()
        plt.subplots_adjust(left=0.25, bottom=0.25)
        a0 = 5
        f0 = 3
        
        ###
        losses = [self.net.learn([fact]) for fact in self.training_data_gen(n)]
        l, = plt.plot(range(len(losses)), losses, lw=2)
        ax.margins(x=0)
        plt.yscale('log')

        axcolor = 'lightgoldenrodyellow'
        axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
        axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

        sfreq = Slider(axfreq, '⍺', 0, 1, valinit=self.net.eta)
        samp = Slider(axamp, 'Num', 1, 1000, valinit=100, valstep=1)
        
        filtfunc = [lambda x:x]
        
        
        big = max(losses)
        ax.set_title(f"maxloss:{big}")
    
        iax = plt.axes([0.025, 0.7, 0.15, 0.15])
        def make_iax_image():
            return np.concatenate([np.concatenate((l.M,np.array([l.b])),axis=0)
                                   for l in self.net.layers
                                  if hasattr(l, 'M')],axis=1)
        def update_iax(img=[iax.imshow(make_iax_image())]):
            img[0].remove()
            img[0] = iax.imshow(make_iax_image())

        def update(val,ax=ax,loc=[l]):
            n = int(samp.val)
            self.net = dill.loads(pickled_net)
            sfunc = lambda x: 2**(-1.005/(x+.005))
            self.net.eta = sfunc(sfreq.val)
            #sfreq.set_label("2.4e"%(self.net.eta,))
            losses = filtfunc[0]([self.net.learn([fact]) for fact in self.training_data_gen(n)])
            big = max(losses)
            ax.set_title(f"⍺={self.net.eta:1.3e},max loss:{big}")
            loc[0].remove()
            loc[0], = ax.plot(range(len(losses)), losses, lw=2,color='xkcd:blue')
            ax.set_xlim((0,len(losses)))
            ax.set_ylim((min(losses),big))
            update_iax()
            fig.canvas.draw_idle()

        sfreq.on_changed(update)
        samp.on_changed(update)

        resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
        button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

    
        def reset(event):
            self.seed += 1
            update()
        button.on_clicked(reset)

        rax = plt.axes([0.025, 0.5, 0.15, 0.15], facecolor=axcolor)
        radio = RadioButtons(rax, ('raw', 'low pass', 'green'), active=0)

        
        def colorfunc(label):
            if label == "raw":
                filtfunc[0] = lambda x:x
            elif label == "low pass":
                filtfunc[0] = lambda x:ndimage.gaussian_filter(np.array(x),3)
            #l.set_color(label)
            #fig.canvas.draw_idle()
            update()
        radio.on_clicked(colorfunc)

        plt.show()
        #return 'gc protect:', update, reset, colorfunc,sfreq,samp, radio, button
        self.gc_protect.append((update, reset, colorfunc,sfreq,samp, radio, button))

## Useful Layers

### Identify

In [39]:
class IdentityLayer(Layer):
    def __call__(self, x):
        return x
    
    def backprop(self, output_delE):
        return output_delE

### Affine
A layer that does an [affine transformation](https://mathworld.wolfram.com/AffineTransformation.html) aka affinity, which is the classic fully-connected layer with output offsets.

$$ \mathbf{M} \mathbf{x} + \mathbf{b} = \mathbf{y} $$
where
$$
\mathbf{x} = \sum_{j=1}^{n} x_j \mathbf{\hat{x}}_j \\
\mathbf{b} = \sum_{i=1}^{m} b_i \mathbf{\hat{y}}_i \\
\mathbf{y} = \sum_{i=1}^{m} y_i \mathbf{\hat{y}}_i
$$
and $\mathbf{M}$ can be written
$$
\begin{bmatrix}
    m_{1,1} & \dots & m_{1,n} \\
    \vdots & \ddots & \vdots \\
    m_{m,1} & \dots & m_{m,n}
\end{bmatrix} \\
$$

#### Error gradient back-propagation
$$ 
\begin{align}
 \frac{\partial loss}{\partial\mathbf{x}}
  = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{x}}
  = \mathbf{M}\frac{\partial loss}{\partial\mathbf{y}}
\end{align}
$$
_SOLVE: Left-multiply or right-multiply?_

#### Parameter adjustment
$$
 \frac{\partial loss}{\partial\mathbf{M}}
 = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{M}}
 = \frac{\partial loss}{\partial\mathbf{y}} \mathbf{x} \\
 \frac{\partial loss}{\partial\mathbf{b}}
 = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{b}}
 = \frac{\partial loss}{\partial\mathbf{y}}
$$

In [40]:
class AffinityLayer(Layer):
    """An affine transformation, which is the classic fully-connected layer with offsets"""
    def __init__(self, n, m):
        self.M = np.empty((m, n))
        self.b = np.empty(m)
        self.randomize()
        
    def randomize(self):
        self.M[:] = np.random.randn(*self.M.shape)
        self.b[:] = np.random.randn(*self.b.shape)
        
    def __call__(self, x):
        self.input = x
        self.output = self.M @ x + self.b
        return self.output
    
    def backprop(self, output_delE):
        input_delE = self.M @ output_delE
        self.M -= np.einsum('i,j', output_delE, self.input) # use np.outer?
        self.b -= output_delE
        return input_delE

### Map
Maps a scalar function on the inputs, for e.g. activation layers.

In [41]:
class MapLayer(Layer):
    """Map a scalar function on the input taken element-wise"""
    def __init__(self, fun, dfundx):
        self.vfun = np.vectorize(fun)
        self.vdfundx = np.vectorize(dfundx)

        
    def __call__(self, x):
        self.input = x
        return self.vfun(x)
    
    def backprop(self, output_delE):
        input_delE = self.vdfundx(self.input) * output_delE
        return input_delE

___

## Tests

### One identity layer
See if the wheels turn:

In [42]:
net = Network()
net.extend(IdentityLayer())
all(net(np.arange(3)) == np.arange(3))

True

It does not learn, as expected:

In [43]:
facts = [(np.arange(2*n, 2*n+2), np.arange(2*n+1, 2*n-1, -1)) for n in range(3)]
net.learn(facts)

1.0

In [44]:
net(np.arange(2,4))

array([2, 3])

### One map layer

In [45]:
net = Network()
net.extend(MapLayer(lambda x: x+1, lambda d: 1))
all(net(np.arange(3)) == np.arange(3)+1)

True

It does not learn, as expected:

In [46]:
net.learn(facts), all(net(np.arange(5)) == np.arange(5)+1), net(np.arange(2,4))

(2.0, True, array([3, 4]))

### One affine layer

In [47]:
net = Network()
net.extend(AffinityLayer(2,2))

In [48]:
t = net.layers[0]
t.M, t.b

(array([[-0.07555974,  0.50894321],
        [-0.79693707,  0.52115632]]),
 array([-0.30574839,  0.99669659]))

#### Can it learn the identity transformation?

In [49]:
bench = NNBench(net)
bench.checkpoint_net()
bench.learning_potential()

20.852776766237184

In [50]:
bench.plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
bench.ideal = lambda v: np.array([v[1], v[0]])
bench.knobs_plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Learn thru a map layer

This layer squares its input and divides by two:

In [52]:
net = Network()
net.extend(AffinityLayer(2,2))

def dtanh(x):
    v = np.tanh(x)
    return (1+v)*(1-v)

net.extend(MapLayer(lambda x:x*x/2.0, lambda d:d))
#net.extend(MapLayer(np.tanh, dtanh))
bench = NNBench(net)
bench.checkpoint_net()

In [53]:
net.layers[0].M, net.layers[0].b

(array([[-0.32158469,  0.15113037],
        [-0.01862772,  0.48352879]]),
 array([0.76896516, 1.36624284]))

#### Can it learn difference squared?

In [54]:
bench.ideal = lambda v: [(v[0]-v[1])**2,0]
#bench.ideal = lambda v: [(v[0]>0)*2-1,(v[0]>v[1])*2-1]
bench.learning_potential()
#bench.knobs_plot_learning(100)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in subtract


nan

In [55]:
bench.knobs_plot_learning(100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in subtract


### add a RELU

In [56]:
bench.net.layers = []
bench.net.extend(AffinityLayer(2,2))
leak = 0
bench.net.extend(MapLayer(lambda x: (x*(1+leak/2)+abs(x)*(1-leak/2))/2, lambda d: [leak,1][1 if d>0 else 0]))
bench.net.layers

### XOR

In [57]:
net = Network()
net.extend(AffinityLayer(2,2))

In [58]:
t = net.layers[0]
t.M, t.b

(array([[ 1.14726479, -0.11022916],
        [ 0.38825041, -0.38712718]]),
 array([-0.58722031,  1.91082685]))